# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
#импорт библиотеки Pandas как pd
data = pd.read_csv('/datasets/data.csv')
# чтение файла data.csv, сохранение его в переменной data
display(data.head(5))
# вывод на экран первых десяти строк из дата-фрейма data


In [ ]:
data.info()
# запрос общей информации о дата-фрейме с помощью метода .info()

In [ ]:
display(data[data['total_income'].isna()].head(5))
# методом isna() найдём все строки с пропусками в столбце total_income и просмотрим первые пять.


In [ ]:
print(data['dob_years'].min())
print(data['dob_years'].max())

# c помощью функций min() и max() проверим минимальный и максимальный возрасты клиентов

**Вывод**

В таблице двеннадцать столбцов. 
Мы наблюдаем вида данных - `float`, `int64` и `object`.

В столбцах 'days_employed' и 'total_income' значения в формате `float`. Также  имеются отрицательные значения в столбце 'days_employed'. В столбце 'dob_years' минимальное значение возраста 0. 

Количество значений в столбцах различается. В столбцах 'days_employed', 'total_income' имеются пропущенные значения. Пропуски в формате `NaN`. 

Данные могли быть пропущены по причине того, что человек не оставил в анкете именно эти данные по причине незнания (ручная ошибка, человеческий фактор), либо была проблема с выгрузкой из системы (системная ошибка). 


## Шаг 2. Предобработка данных

### Обработка пропусков

В столбцах 'days_employed', 'total_income' имеются пропущенные значения. Пропуски в формате `NaN`.
Примем гипотизу о том, что в банк за кредитом не стали бы обращаться люди не имеющие никакого дохода а также не работающие ни одного дня. 
Следовательно каждому пропущенному значению сопоставим медианное в зависимости от типа дохода.
Заменим отрицательные значения в столбце 'days_employed'. 
Заменим нулевые значения в столбце 'dob_years' - средним значением.

In [ ]:
income_unique_type = data['income_type'].unique()
#создадим список, в котором хранятся уникальные значения столбца 'income_type'

#сопоставим каждому пропущенному значению в столбце 'income_type' - медианное, в зависимости от типа дохода:
for unique_type in income_unique_type:
    median_total_income_by_type = data.loc[data['income_type'] == unique_type]['total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['income_type'] == unique_type),'total_income']=median_total_income_by_type

#сопоставим каждому пропущенному значению в столбце 'days_employed' - медианное, в зависимости от типа дохода:
for unique_type in income_unique_type:
    median_total_days_by_type = data.loc[data['income_type'] == unique_type]['days_employed'].median()
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == unique_type),'days_employed']=median_total_days_by_type


data['days_employed'] = data['days_employed'].abs()
#заменили отрицательные значения в столбце'days_employed'


data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean() 
#заменили нулевые значения в возрасте - средним значением

In [ ]:
data.info()
# запрос общей информации о дата-фрейме с помощью метода .info()

**Вывод**

Предобработка обнаружила следующие проблемы в данных:
- в столбцах 'days_employed', 'total_income' имелись пропущенные значения в формате `NaN`,
- наличие отрицательных значений в столбце'days_employed',
- нулевые значения в столбце 'dob_years'.

Пропущенные значения мы заменили на медианные, отрицательные на положительные. Нулевые заменили на средние.


### Замена типа данных

В столбцах 'days_employed', 'total_income', 'dob_years' видим тип данных `float`. 
Для удобства представления данных а также расчетов, с целью избежания больших чисел после запятой. заменим тип данных на `int`.

In [ ]:
data = data.astype({'days_employed': 'int64', 'total_income': 'int64', 'dob_years': 'int64',})

In [ ]:
data.info()
# запрос общей информации о дата-фрейме с помощью метода .info()

**Вывод**


Мы заменили вещественный тип данных в столбцах 'days_employed', 'total_income', 'dob_years'  на целочисленный.

### Обработка дубликатов

In [ ]:
print(data['education'].unique())
#с помощью метода unique() посмотрим уникальные значения в столбце 'education'

Мы видим дубликаты в столбце education.
Имеются следующие уровни образования: начальное, среднее, высшее, неоконченное высшее а также ученая степень. У каждого наименования присутствуют разные регистры букв. Необходимо заменить буквы на строчные.

Также проверим наличие дубликатов в дата-фрейме, при наличии - избавимся от них.

In [ ]:
data['education'] = data['education'].str.lower()
# с помощью метода str.lower() приведем значения в столбце 'education' к нижнему регистру
print(data['education'].unique())


print(data.duplicated().sum())
#проверим наличие дубликатов в дата-фрейме
data = data.drop_duplicates().reset_index()
#избавляемся от дубликатов методом drop_duplicates() вместе с reset_index
print(data.duplicated().sum())

**Вывод**

Наименования данных в столбце об образовании привели к нижнему регистру, избавились от дубликатов в дата-фрейме.

### Лемматизация

In [ ]:
print(data['purpose'].unique())

В столбце 'purpose', где описаны цели получения кредита, видим, что цели получения кредита можно распределить на 4 группы: свадьба, недвижимость, автомобиль, образование.

Сделаем это с помощью лемматизации.

In [ ]:
from pymystem3 import Mystem
m = Mystem()
#импортируем библиотеку pymystem3 с фунцкцией лемматизации
data['lemmas_purpose'] = data['purpose'].apply(m.lemmatize)
#лемматизируем столбец 'purpose' применяя функцию m.lemmatize и сохраняем данные в новом столбце - 'lemmas_purpose'

#создадим функцию, которая лемматизирует столбец 'purpose', проверит наличие ключевых слов и исходя из них присвоит им группу 
def group_lemmas(purpose):
    lemmas_row = m.lemmatize(purpose)
    for lemma in lemmas_row:
        if 'свадьба' in lemma:
            return 'свадьба'
        if 'жилье' in lemma or 'недвижимость' in lemma:
            return 'недвижимость'
        if 'автомобиль' in lemma:
            return 'автомобиль'
        if 'образование' in lemma:
            return 'образование'
        
data['purpose_groups'] = data['purpose'].apply(group_lemmas)    

display(data.head(5))

**Вывод**

После просмотра данных в столбце с целью получения кредита, мы решили, что в целом их можно разбить на несколько групп:
- свадьба
- недвижимость
- автомобиль
- образование

Данное разделение осуществили с помощью процесса лемматизации. Выделили в каждой цели леммы, и в дальнейшем, с помощью функции - присвоили соответсвующие группы.

### Категоризация данных

1. Возраст клиентов. Распределим клиентов в зависимости от возраста на три группы:
- молодой
- среднего возраста
- пожилой

Далее посмотрим существует ли зависимость между наличием задолженности и возрастом.

In [ ]:
#проверим минимальный и максимальный возраст клиентов
# print(data['dob_years'].min())  #минимальный возраст 19
# print(data['dob_years'].max())  #максимальный возраст 75

#напишем функцию, которая в зависимости от возраста клиента - отнесет клиента к определенной категории
def age_group(age):
    if 18 <= age <= 35:
        return 'молодой'
    if 35 < age <= 55:
        return 'среднего возраста'
    return 'пожилой'

data['age_group'] = data['dob_years'].apply(age_group)



2. Количество детей. Распределим клиентов на три группы в зависимости от количества детей:
- один ребенок
- два и более детей
- нет детей

In [ ]:
#print (data['children'].value_counts()) # у 76 людей наблюдается 20 детей, у 47 по -1 ребенку
data['children'] = abs(data['children']) #заменим количество -1 на 1, возможно была проблема с выгрузкой данных

# для более корректного анализа заменим 20 детей на медианное значение,, округлим и изменим тип данных на целочисленные (дети являются целыми объектами, половины быть не может)
data.loc[data['children'] == 20, 'children'] = data['children'].median().round().astype('int') 
    
#print (data['children'].value_counts()) #теперь максимальное количество детей 5

# напишем функцию, которая в зависимости от количества людей - отнесет клиента к определенной категории
def having_kids(number_of_children):
    if number_of_children == 1:
        return 'один ребенок'
    if number_of_children >1:
        return 'два и более детей'
    return 'нет детей'

data['children_category'] = data['children'].apply(having_kids)
#display(data.head(5))



3. Уровень дохода. Распределим клиентов банка по уровням доходов. Мы видим следующие типы занятости:
- сотрудник
- компаньон
- госслужащий
- предприниматель
- пенсионер
- в декрете
- студент
- безработный

Практически в каждом виде занятости видим, что по уровням доходов можно разбить клиентов на несколько групп:
- низкий уровень дохода 
- средний уровень дохода
- состоятельные люди
- богатые

Безработных, в декрете, студентов а также предпринимателей можно исключить из анализа в виду ограниченного набора данных (по 1 или 2 клиенту на каждую группу).

In [ ]:
#в каждой группе по типу занятости посмотрим минимальное,максимальное и медианное значение дохода для дальшейшего распределения по групам
unique_income_type_groups = data['income_type'].unique()

for group in unique_income_type_groups:
    min_value_of_income= data[data['income_type'] == group]['total_income'].min()
    max_value_of_income= data[data['income_type'] == group]['total_income'].max()
    median_value_of_income= data[data['income_type'] == group]['total_income'].median()
    #print(group, min_value_of_income, max_value_of_income, median_value_of_income)


In [ ]:
#напишем функцию, которая в зависимости от уровня дохода - отнесет клиента к определенной категории
#отнесем людей с уровне дохода до 30 тыс. руб. к людям с низким уровнем дохода, 
#от 30 до 90 тыс. руб к людям со средним доходом, 
# от 90 до 200 тыс. руб. к состоятельным, от 200 и выше к богатым

def income_group(income):
    if income <= 30000:
        return 'низкий уровень дохода'
    if 30000 < income <= 100000:
        return 'средний уровень дохода'
    if 100000 < income <= 200000:
        return 'состоятельные'        
    return 'богатые'        
    
data['income_category'] = data['total_income'].apply(income_group)  
#display(data.head(5))

        

**Вывод**

Мы распределили данные по категориям: по возрасту, количеству детей а также уровню доходов исходя из типа получения дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между возрастом и возвратом кредита в срок?

In [ ]:
data_pivot_age = data.pivot_table('debt', index='age_group', aggfunc=['count','sum','mean'])
display(data_pivot_age) 


**Вывод**

Исходя из результатов видим, что чем старше люди - тем меньше задолженностей. Банку предпочтительнее выдавать кредиты людям более старшего возраста, нежели молодым.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_pivot_children = data.pivot_table('debt', index='children_category', aggfunc=['count','sum','mean'])
display(data_pivot_children) 


**Вывод**

Чем больше детей у клиента - тем больше риск невозврата кредита. Причиной могут быть риски, связанные с дополнительными тратами, которые люди с детьми могут понести.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot_family = data.pivot_table('debt', index='family_status', aggfunc=['count','sum','mean'])
display(data_pivot_family) 

**Вывод**

Семейным людям кредиты дают охотнее, нежели тем, кто не замужем/женат, либо находится в гражданском браке. Вдовцам а также людям в разводе кредиты также дают чаще. Возможно, это определяется степенью ответсвенности людей в той или иной категории.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#вначале отфильтруем данные, т.е будем рассматривать для анализа только следующие типы занятости:
# госслужащие, компаньоны, пенсионеры и сотрудники 
#(безработных, в декрете, студентов и предпринимателей исключаем ввиду небольшого количества позиций на каждый тип (1-2))
data_income = data[(data['income_type'] == 'сотрудник') | (data['income_type'] == 'пенсионер') | (data['income_type'] == 'госслужащий') | (data['income_type'] == 'компаньон')]
data_pivot_income_type = data_income.pivot_table('debt', index='income_type', columns='income_category', aggfunc=['count','sum','mean'])
display(data_pivot_income_type)

**Вывод**

Если рассматривать результаты, начиная со среднего уровня заработка (в категорию бедные скорее всего попало небольшое количество элементов выборки) - пенсионеры являются самыми отвественными клиентами и возвращают кредиты лучше всех. Далее идут госслужащие, затем компаньоны и сотрудники. 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot_purpose = data.pivot_table('debt', index='purpose_groups', aggfunc=['count','sum','mean'])
display(data_pivot_purpose) 

**Вывод**

Лучше всего кредиты возвращают клиенты у кого целью получения кредита является покупка недвижимости. Далее идет свадьба. Реже выплачивают кредиты клиенты целью которых является покупка автомобиля.

## Шаг 4. Общий вывод

Проанализировав данные можем сделать портрет идеального клиента с максимальной вероятностью возврата кредита:
- возраст: средний либо более старший
- тип занятости: пенсионер либо работающий госслужащий
- наличие детей: не имеющий детей 
- семейное положение: семейный человек, либо в разводе, либо вдовец
- цель получения: недвижимость либо свадьба